In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pickle
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import activations
from tensorflow.keras import metrics
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler


In [3]:

# inv_id = np.load('../input/ubiquant-train-data/inv_id.npy')
# inv_encoder = keras.layers.experimental.preprocessing.IntegerLookup()
# inv_encoder.adapt(inv_id)
# inv_vocb = len(inv_encoder.get_vocabulary())

In [4]:

data = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
print(data.shape)

data = data.drop(columns=['time_id', 'row_id', 'investment_id', 'target']).values
scaler = StandardScaler()
scaler.fit(data)
data = None

(3141410, 304)


In [5]:
input_dim = 300

In [6]:
def build_encoder(input_dim, noise_rate=0.05):
    ori_input = Input(shape=(input_dim,), dtype='float16')
    encode = layers.BatchNormalization()(ori_input)
    encode = layers.GaussianNoise(stddev=noise_rate)(encode)
    encode = layers.Dense(256)(encode)
    encode = Mish()(encode)
    decode = layers.Dropout(0.1)(encode)
    decode = layers.Dense(input_dim, name='decoded')(decode)
    x = layers.Dense(265, use_bias=False)(decode)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(1, name='target')(x)

    encoder = models.Model(inputs=[ori_input], outputs=[encode])
    autoencoder = models.Model(inputs=[ori_input], outputs=[decode, x])
    return autoencoder, encoder

In [7]:
class Mish(layers.Layer):
    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)

    def call(self, inputs, *args, **kwargs):
        return inputs * K.tanh(K.softplus(inputs))

In [8]:
class Squeeze(layers.Layer):
    def __init__(self, **kwargs):
        super(Squeeze, self).__init__(**kwargs)

    def call(self, inputs, *args, **kwargs):
        return K.squeeze(inputs, axis=1)

In [9]:
def build_model(encoder):
    ox_input = Input(shape=(input_dim,), dtype='float16')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128)(x)
    x = Mish()(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])
    return model

In [10]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [11]:
weights = ['../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-1-ep8-loss0.8170.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-10-ep8-loss0.8171.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-11-ep8-loss0.8174.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-12-ep8-loss0.8181.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-2-ep8-loss0.8170.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-3-ep8-loss0.8181.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-4-ep8-loss0.8191.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-5-ep8-loss0.8176.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-6-ep8-loss0.8170.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-7-ep8-loss0.8175.h5',
          '../input/ubiquantbigdropbt512weights/origin-bigdrop-bt512-test-9-ep8-loss0.8182.h5',
          
           '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-1-ep8-loss0.8173.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-10-ep8-loss0.8175.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-11-ep8-loss0.8181.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-12-ep8-loss0.8185.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-2-ep8-loss0.8176.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-3-ep8-loss0.8182.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-4-ep8-loss0.8178.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-5-ep8-loss0.8183.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-6-ep8-loss0.8185.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-7-ep8-loss0.8171.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-8-ep8-loss0.8176.h5',
          '../input/ubiquantbigdrop1bt512weights/origin-bigdrop-1-bt512-test-9-ep8-loss0.8172.h5']

model = []
for w in weights:
    autoencoder, encoder = build_encoder(input_dim)
    encoder.trainable = False
    model_0 = build_model(encoder)
    model_0.load_weights(w)
    model.append(model_0)
print(len(model))

2022-04-18 03:37:04.703429: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 03:37:04.823103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 03:37:04.823906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 03:37:04.825103: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

23


In [12]:
# an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    data_ox = test_df.drop(columns=['row_id', 'investment_id']).values
    data_ox = scaler.transform(data_ox)
    temp = model[0].predict(data_ox, batch_size=2048)
    for i in range(1, len(model)):
        temp = temp + model[i].predict(data_ox, batch_size=2048)
    print(temp)
    sample_prediction_df['target'] = temp
    env.predict(sample_prediction_df)   # register your predictions

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


2022-04-18 03:37:14.145735: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


[[-1.0057709 ]
 [-0.22479707]]
[[-1.0372939 ]
 [-0.65028423]
 [-0.8674945 ]]
[[-0.5857274 ]
 [ 0.05598549]
 [-0.85898656]]
[[-0.47927392]]
